<a href="https://colab.research.google.com/github/jc890/python/blob/master/Audio_analysis_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Audio + NLP Event Detection Module
---------------------------------
Responsibilities:
- Extract audio from video
- Transcribe speech using Whisper
- Detect conversational events:
    - Question
    - Agreement
    - Disagreement

This module determines *when* important events occur.
"""

import subprocess
import os
import re
import whisper

# =========================
# CONFIGURATION
# =========================
AUDIO_PATH = "workspace/audio.wav"

# Simple keyword lists (safe, explainable NLP)
AGREEMENT_KEYWORDS = [
    "yes", "yeah", "exactly", "right", "correct", "i agree", "absolutely"
]

DISAGREEMENT_KEYWORDS = [
    "no", "not really", "i disagree", "i don't think", "that's not", "wrong"
]

QUESTION_WORDS = [
    "what", "why", "how", "when", "where", "who", "do you", "can you"
]


# =========================
# AUDIO EXTRACTION
# =========================
def extract_audio(video_path, audio_path=AUDIO_PATH):
    """
    Extracts mono 16kHz WAV audio from video using ffmpeg.
    """

    command = [
        "ffmpeg",
        "-y",
        "-i", video_path,
        "-ac", "1",
        "-ar", "16000",
        audio_path
    ]

    subprocess.run(
        command,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True
    )

    return audio_path


# =========================
# NLP EVENT DETECTION
# =========================
def classify_text(text):
    """
    Classify text into conversational event types.
    """

    text_lower = text.lower()

    # Question detection
    if "?" in text_lower:
        return "Question"

    for q in QUESTION_WORDS:
        if q in text_lower:
            return "Question"

    # Agreement detection
    for word in AGREEMENT_KEYWORDS:
        if word in text_lower:
            return "Agreement"

    # Disagreement detection
    for word in DISAGREEMENT_KEYWORDS:
        if word in text_lower:
            return "Disagreement"

    return None


# =========================
# MAIN AUDIO + NLP PIPELINE
# =========================
def detect_audio_events(video_path):
    """
    Extracts audio, transcribes speech, and detects conversational events.

    Returns:
    [
        {
            "event": "Agreement",
            "timestamp": 60.5,
            "text": "yes exactly"
        }
    ]
    """

    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video not found: {video_path}")

    # Step 1: Extract audio
    audio_file = extract_audio(video_path)

    # Step 2: Load Whisper model (base = safe choice)
    model = whisper.load_model("base")

    # Step 3: Transcribe audio
    result = model.transcribe(audio_file)

    events = []

    for segment in result["segments"]:
        text = segment["text"].strip()
        start_time = segment["start"]

        event_type = classify_text(text)

        if event_type:
            events.append({
                "event": event_type,
                "timestamp": float(start_time),
                "text": text
            })

    return events


# =========================
# TEST BLOCK
# =========================
if __name__ == "__main__":
    print("Running Audio + NLP self-test...")

    TEST_VIDEO = "workspace/input.mp4"

    if not os.path.exists(TEST_VIDEO):
        print("Test video not found. Place a video at workspace/input.mp4")
    else:
        detected_events = detect_audio_events(TEST_VIDEO)

        if not detected_events:
            print("No conversational events detected.")
        else:
            for e in detected_events:
                print(
                    f"[{e['event']}] at {e['timestamp']:.2f}s → {e['text']}"
                )

Running Audio + NLP self-test...
Test video not found. Place a video at workspace/input.mp4


In [ ]:
!pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=9406635882bbc06aa377fe5ddf3c65f123a1ee2615f1f2c7f08e70408cfb74f2
  Stored in directory: /root/.cache/pip/wheels/34/b8/4e/9c4c3351d670e06746a340fb4b7d854c76517eec225e5b32b1
Successfully built whisper
